In [4]:
import torch
from torchvision import models, transforms
from PIL import Image
import os

# Define the device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Model settings
IMG_SIZE = 224  # Input size for the model
NUM_CLASSES = 8  # Number of output classes (as per your training setup)

# Load the fine-tuned model
model = models.densenet121(pretrained=True)

# Adjust the classifier to match your fine-tuning architecture
model.classifier = torch.nn.Sequential(
    torch.nn.Linear(model.classifier.in_features, 512),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(512, NUM_CLASSES)
)

# Load the model weights
model.load_state_dict(torch.load('./Models/best_fine_tuned_model.pth'))  # Ensure the model path is correct
model = model.to(device)
model.eval()

# Define the image transformation (same as during training)
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Function to predict the class of an input image
def predict_image(image_path):
    # Open image
    image = Image.open(image_path).convert('RGB')
    
    # Apply transformations
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted_class = outputs.max(1)
    
    return predicted_class.item()

Using device: cuda


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_10192\2032221645.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code duri

In [33]:
import pandas as pd

# Load the mapping table
def load_mapping_table():
    return pd.read_csv('soil_crop_mapping.csv')

# Function to recommend crops with detailed explanations
def recommend_crops(soil_type):
    # Load the mapping table
    mapping_df = load_mapping_table()
    
    # Find the row corresponding to the predicted soil type
    soil_data = mapping_df[mapping_df['Soil Type'] == soil_type]
    
    if not soil_data.empty:
        # Extract nutrient and crop information
        nutrients = soil_data.iloc[0]
        recommended_crops = nutrients['Recommended Crops']
        
        # Prepare the base output
        print(f"Soil Type: {soil_type}")
        print(f"Nutrient Profile:")
        print(f"  - Nitrogen (N): {nutrients['Nitrogen (N)']}")
        print(f"  - Phosphorus (P): {nutrients['Phosphorus (P)']}")
        print(f"  - Potassium (K): {nutrients['Potassium (K)']}")
        print(f"  - Calcium (Ca): {nutrients['Calcium (Ca)']}")
        print(f"  - Magnesium (Mg): {nutrients['Magnesium (Mg)']}")
        print(f"  - Iron (Fe): {nutrients['Iron (Fe)']}")
        print(f"  - Organic Matter: {nutrients['Organic Matter']}")
        print(f"\nRecommended Crops: {recommended_crops}")
        
        # Append reasoning for the recommendation
        explanations = {
            "Alluvial Soil": (
                "Alluvial soils are rich in humus, well-draining, and have excellent fertility. "
                "These soils support crops requiring deep root penetration and high water retention. "
                "Rice thrives due to its water-intensive nature, while wheat and maize benefit from the high nutrient availability."
            ),
            "Black Soil": (
                "Black soil, also known as Regur soil, is clayey, moisture-retentive, and rich in calcium carbonate and magnesium. "
                "This makes it ideal for cotton, often referred to as 'black gold,' and other dryland crops like sorghum and millet. "
                "Its high cation-exchange capacity enhances crop nutrient uptake."
            ),
            "Cinder Soil": (
                "Cinder soils, formed from volcanic ash, have good aeration and drainage but are low in organic content. "
                "They support crops like maize and sorghum, which are drought-resistant and thrive in less fertile conditions. "
                "Pineapple benefits from the soil's aeration, promoting root health."
            ),
            "Clay Soil": (
                "Clay soils are nutrient-rich but have poor drainage, making them prone to waterlogging. "
                "Crops like rice and sugarcane, which tolerate or thrive in saturated conditions, are ideal. "
                "The soil's fine texture helps retain essential plant nutrients like potassium and phosphorus."
            ),
            "Laterite Soil": (
                "Laterite soils are acidic, iron-rich, and nutrient-depleted due to leaching. "
                "Perennial crops like tea, coffee, and cashew nuts thrive in these conditions, as they adapt well to acidic and well-draining soils. "
                "Rubber cultivation also benefits from its porosity."
            ),
            "Peat Soil": (
                "Peat soils are highly organic and acidic, making them ideal for root crops like potatoes and carrots. "
                "These soils are naturally waterlogged, supporting crops that require consistent moisture. "
                "The high organic content also supports small-scale berry farming."
            ),
            "Red Soil": (
                "Red soils, rich in iron but low in nitrogen and phosphorus, are ideal for hardy crops. "
                "Millet, groundnuts, and pulses thrive due to their adaptability to low fertility and drought resistance. "
                "These crops also help in nitrogen fixation, improving soil fertility over time."
            ),
            "Yellow Soil": (
                "Yellow soils are moderately rich in magnesium and iron but have limited nitrogen availability. "
                "Groundnuts and pulses, which have low nitrogen demand and can fix atmospheric nitrogen, are ideal. "
                "Cassava and maize, known for their adaptability, also perform well in these moderately fertile soils."
            )
        }
        
        if soil_type in explanations:
            print(f"\n{explanations[soil_type]}")
    else:
        print("Soil type not found in the mapping table.")


In [34]:
class_names = ['Alluvial Soil', 'Black Soil', 'Cinder Soil', 'Clay Soil', 'Laterite Soil', 'Peat Soil', 'Red Soil', 'Yellow Soil']
image_path = './Testing/Red.jpeg'  
predicted_class = predict_image(image_path)
predicted_class_name = class_names[predicted_class]
recommend_crops(predicted_class_name)

Soil Type: Red Soil
Nutrient Profile:
  - Nitrogen (N): Low
  - Phosphorus (P): Low
  - Potassium (K): Low
  - Calcium (Ca): Low to Moderate
  - Magnesium (Mg): Low
  - Iron (Fe): High
  - Organic Matter: Low

Recommended Crops: Millet, Groundnuts, Cotton, Pulses

Red soils, rich in iron but low in nitrogen and phosphorus, are ideal for hardy crops. Millet, groundnuts, and pulses thrive due to their adaptability to low fertility and drought resistance. These crops also help in nitrogen fixation, improving soil fertility over time.
